Import Statements

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torch.utils.data.dataset import Subset
from torchvision import datasets, models, transforms
import torch.nn.functional as F

Device Configuration

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda:0


Data Prep

In [5]:
# Download and load the CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_set = datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)

# Split train data into train and validation
train_size = int(0.8 * len(train_set))
val_size = len(train_set) - train_size
train_dataset, val_dataset = random_split(train_set, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=4, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


CNN Model Architecture

In [6]:
# Define the CNN Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)

        # Batch normalization layers
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        # Max pooling
        self.pool = nn.MaxPool2d(2, 2)

        # Dropout
        self.dropout = nn.Dropout(0.2)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)  # 10 classes in CIFAR-10

    def forward(self, x):
        # Convolutional layers with ReLU and batch normalization
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        # Flatten the tensor for the fully connected layers
        x = x.view(-1, 128 * 4 * 4)

        # Fully connected layers with dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to(device)

Loss and Optimizer

In [7]:
# 4. Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

Training Loop

In [8]:
# Function for training
def train(net, train_loader, val_loader, criterion, optimizer, epochs=20):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        train_losses.append(running_loss / len(train_loader))
        val_loss = validate(net, val_loader, criterion)
        val_losses.append(val_loss)
        print(f'Epoch {epoch + 1}, Train Loss: {train_losses[-1]}, Validation Loss: {val_losses[-1]}')

    return train_losses, val_losses

# Function for validation
def validate(net, val_loader, criterion):
    total_loss = 0.0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    return total_loss / len(val_loader)

# Train the network
train_losses, val_losses = train(net, train_loader, val_loader, criterion, optimizer)

# Plot train loss vs val loss
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1, Train Loss: 1.512519698060304, Validation Loss: 1.1980904674127697
Epoch 2, Train Loss: 1.1190507774993776, Validation Loss: 1.0470456995368005
Epoch 3, Train Loss: 0.9561110649188049, Validation Loss: 0.9785899772398173
Epoch 4, Train Loss: 0.8514268781432882, Validation Loss: 0.9415240365795791
Epoch 5, Train Loss: 0.7684248074375472, Validation Loss: 0.9235831525850575
Epoch 6, Train Loss: 0.7106433897865452, Validation Loss: 0.9440448384734104
Epoch 7, Train Loss: 0.6589279781563979, Validation Loss: 0.8998848186862423
Epoch 8, Train Loss: 0.6188186427894813, Validation Loss: 0.8936783992060635
Epoch 9, Train Loss: 0.5767090888988954, Validation Loss: 0.9096712813504506


KeyboardInterrupt: 

Test Evaluation

In [ ]:
# Evaluate on test data and plot the test error vs epochs
test_error = []
with torch.no_grad():
    for epoch in range(200):
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        error = 1 - correct / total
        test_error.append(error)
        print(f'Epoch {epoch + 1}, Test Error: {error}')

plt.plot(test_error, label='Test Error')
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.legend()
plt.show()

In [ ]:
import os
import torch

# Define the directory path
directory_path = './test_errors/'

# Create the directory if it doesn't exist
os.makedirs(directory_path, exist_ok=True)

# Save the test errors to a file
torch.save(test_error, directory_path + 'DF_CNN_Adam_test_errors.pt')  # Saving the list as a .pt file

# Providing the path where the file is saved
path_to_saved_file = directory_path + 'DF_CNN_Adam_test_errors.pt'
path_to_saved_file
